In [12]:
# try
import sys

import pandas as pd # type: ignore
import numpy as np # type: ignore

import matplotlib.pyplot as plt # type: ignore
import seaborn as sns # type: ignore

import networkx as nx # type: ignore
import graphviz # type: ignore

import statsmodels.formula.api as smf # type: ignore
import statsmodels.api # type: ignore
from statsmodels.miscmodels.ordinal_model import OrderedModel # type: ignore

from dowhy import CausalModel, gcm # type: ignore

from sklearn.linear_model import LogisticRegression # type: ignore
from sklearn.neighbors import NearestNeighbors # type: ignore

import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [13]:
df = pd.read_csv("../data/processed/data.csv")
df.shape

In [10]:
nodes_list = [
'Rating',
'Product_category_encoded',
'freight_value',
'distance_km',
'season',
'is_delivery_late'
]

df[nodes_list].isnull().sum()

Rating                      948
Product_category_encoded      0
freight_value                 0
distance_km                   0
season                        0
is_delivery_late              0
dtype: int64

In [ ]:
# Temporary fix for imputing rating
# df['Rating'].fillna(df['Rating'].mean(), inplace=True)

# Potential Outcomes

In [7]:
# Matteo